In [33]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os

credit_keywords = {
    "zelle": "Gross Trucking Income",
    "deposit": "Gross Trucking Income",
    "payment received": "Gross Trucking Income",
    "refund": "Reimbursement",
    "rebate": "Reimbursement",
    "mobile deposit": "Gross Trucking Income",
    "purchase return" : "return",
}

credit_return_keywords = {
    "fontana ca" : "Subhaulers & Carriers"
}

debit_keywords = {
    "zelle": "Gross Trucking Income",
    "fuel": "Fuel Cost",
    "pump": "Fuel Cost",
    "gas": "Fuel Cost",
    "insurance": "Insurance",
    "advance": "Loan or Advance",
    "loan": "Loan or Advance",
    "restaurant": "Food & Dining",
    "bar": "Entertainment",
    "grill": "Food & Dining",
    "hotel": "Lodging",
    "subscription": "Subscription",
    "grocery": "Groceries",
    "prepass": "Tolls and Parking",

    
    "purchase authorized": "Subhaulers & Carriers",
    "check" : "Truck Maintenance Cost",
    

    # Companies
    "pathward na" : "Liability Insurance",
    "ibex " : "Liability Insurance",

    # Licences and Permits
    "jst registration" : "Business Licenses and Permits",
    "unified carrier" : "Business Licenses and Permits",

    # Medical
    "drug testing": "Drug Testing",

    # Taxes
    "cdtfa" : "CDTFA",
    "tax" : "Taxes",

    # Bank
    "monthly service fee" : "Bank Service Charges"
}
debit_check_keywords = {
    "check" : "Truck Maintenance Cost",
    "cashed" : "Truck Maintenance Cost",
    "deposited" : "Truck Maintenance Cost",
    "overdraft fee": "Bank Service Charges",
}


def process_excel(input_path, output_folder):
    try:
        df = pd.read_excel(input_path)
        categories = []

        for _, row in df.iterrows():
            amount = row["Amount"]
            desc = row["Description"]

            if amount > 0:
                category = categorize_credit(desc)
            else:
                category = categorize_debit(desc)

            # if not category:
            #     category = lookup_vendor_web(desc)

            categories.append(category)

        df["Category"] = categories

        output_path = os.path.join(output_folder, f"processed_{os.path.basename(input_path)}")
        df.to_excel(output_path, index=False)
    except Exception as e:
        messagebox.showerror("Processing Error", f"Failed processing:\n{input_path}\n\nError:\n{str(e)}")


def categorize_credit(description):
    desc = str(description).lower()
    for key, value in credit_keywords.items():
        if key in desc:
            if value == "return":
                for key1, value1 in credit_return_keywords.items():
                    if key1 in desc:
                        return value1
            return value
            
    return None

def categorize_debit(description):
    desc = str(description).lower()
    for key, value in debit_keywords.items():
        if key in desc:
            if "check" in desc:
                for key1, value1 in debit_check_keywords.items():
                    if desc.startswith(key1):
                        return value1
            return value
    return None

def select_files():
    files = filedialog.askopenfilenames(filetypes=[("Excel files", "*.xlsx *.xls")])
    if files:
        global selected_input_files
        selected_input_files = list(files)  # store real list
        input_files_display_var.set("; ".join(files))  # only for display

def select_output_folder():
    folder = filedialog.askdirectory()
    if folder:
        output_folder_var.set(folder)

def run_processing():
    if not selected_input_files or not output_folder_var.get():
        messagebox.showerror("Error", "Please select input files and output folder.")
        return
    for file_path in selected_input_files:
        process_excel(file_path, output_folder_var.get())
    messagebox.showinfo("Success", "Processing completed!")

root = tk.Tk()
root.title("Excel Processor")

# Variables
selected_input_files = []  # real list of files # JUST for Debugging
selected_input_files.append("C:/Users/TCE/Downloads/Input.xlsx")  # JUST for Debugging
input_files_display_var = tk.StringVar()  # shown in entry
input_files_display_var.set("; ".join(selected_input_files))  # JUST for Debugging
output_folder_var = tk.StringVar()
output_folder_var.set("C:/Users/TCE/Downloads")  # JUST for Debugging

# Widgets
tk.Button(root, text="Select Excel Files", command=select_files).pack(pady=5)
tk.Entry(root, textvariable=input_files_display_var, width=80, state="readonly").pack(pady=5)

tk.Button(root, text="Select Output Folder", command=select_output_folder).pack(pady=5)
tk.Entry(root, textvariable=output_folder_var, width=80, state="readonly").pack(pady=5)

tk.Button(root, text="Run", command=run_processing).pack(pady=20)

root.mainloop()
